In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Load dataset
df = pd.read_csv("index_finger_keypoints.csv")

# Features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Encode class labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(np.unique(y_encoded))

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [3]:
print(np.unique(y))

['jump' 'slide' 'swipe_left' 'swipe_right']


In [4]:
# Create DataLoader
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [5]:
# Define the model
class GestureClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(GestureClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Initialize model, loss, optimizer
model = GestureClassifier(input_size=10, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Training loop with validation
epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    # Training Phase
    model.train()
    total_loss = 0
    correct = 0
    total_train = 0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_X.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_y).sum().item()
        total_train += batch_y.size(0)

    train_accuracy = correct / total_train
    avg_train_loss = total_loss / total_train

    # Validation Phase
    model.eval()
    val_loss = 0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for val_X, val_y in test_loader:
            val_X, val_y = val_X.to(device), val_y.to(device)

            val_outputs = model(val_X)
            loss = criterion(val_outputs, val_y)

            val_loss += loss.item() * val_X.size(0)
            _, val_predicted = torch.max(val_outputs, 1)
            val_correct += (val_predicted == val_y).sum().item()
            total_val += val_y.size(0)

    val_accuracy = val_correct / total_val
    avg_val_loss = val_loss / total_val

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f} | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")


Epoch 1/50 | Train Loss: 1.2095 | Train Acc: 0.5274 | Val Loss: 1.0366 | Val Acc: 0.7392
Epoch 2/50 | Train Loss: 0.8072 | Train Acc: 0.8575 | Val Loss: 0.5643 | Val Acc: 0.9295
Epoch 3/50 | Train Loss: 0.3656 | Train Acc: 0.9589 | Val Loss: 0.2088 | Val Acc: 0.9815
Epoch 4/50 | Train Loss: 0.1435 | Train Acc: 0.9839 | Val Loss: 0.0955 | Val Acc: 0.9876
Epoch 5/50 | Train Loss: 0.0736 | Train Acc: 0.9898 | Val Loss: 0.0576 | Val Acc: 0.9913
Epoch 6/50 | Train Loss: 0.0467 | Train Acc: 0.9926 | Val Loss: 0.0411 | Val Acc: 0.9926
Epoch 7/50 | Train Loss: 0.0340 | Train Acc: 0.9944 | Val Loss: 0.0327 | Val Acc: 0.9951
Epoch 8/50 | Train Loss: 0.0268 | Train Acc: 0.9951 | Val Loss: 0.0262 | Val Acc: 0.9951
Epoch 9/50 | Train Loss: 0.0232 | Train Acc: 0.9960 | Val Loss: 0.0215 | Val Acc: 0.9963
Epoch 10/50 | Train Loss: 0.0197 | Train Acc: 0.9960 | Val Loss: 0.0209 | Val Acc: 0.9963
Epoch 11/50 | Train Loss: 0.0174 | Train Acc: 0.9960 | Val Loss: 0.0170 | Val Acc: 0.9963
Epoch 12/50 | Train

In [7]:
# Save model and label encoder classes
torch.save(model.state_dict(), "gesture_classifier_model.pth")
np.save("gesture_classes.npy", label_encoder.classes_)
print("✅ Model and class labels saved.")
import joblib
joblib.dump(scaler, 'scaler.pkl')

✅ Model and class labels saved.


['scaler.pkl']